In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timeit

## Algorithm 1 

In [54]:
assignments=[]
def cross(A, B):
    return [a+b for a in A for b in B]

rows = 'ABCDEFGHI'
cols = '123456789'

boxes = cross(rows, cols)
assert  len(boxes) == 81

rows_units = [cross(row, cols) for row in rows]
cols_units = [cross(rows, number) for number in cols]
square_units = [cross(a, b) for a in ('ABC', 'DEF', 'GHI') for b in ('123', '456', '789')]

unitlist = rows_units + cols_units + square_units

units = {box: [unit for unit in unitlist if box in unit] for box in boxes}


peers = {box: set(sum(units[box],[])) - set([box]) for box in boxes}

def assign_value(values, box, value):
  
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

def algo(values):
         for key in boxes:
          if len(values[key]) == 2:
             part_row = rows_units[rows.index(key[0])]
             part_colum = cols_units[cols.index(key[1])]
             part_sqers = []
             for sqare in square_units:
                 if key in sqare:
                     part_sqers = sqare
                     break

           
             row_twin = False
             colum_twin = False
             sqr_twin = False
             
             for i in range(len(part_row)):
                 if part_row[i] != key:
                     if values[part_row[i]] == values[key]:
                         row_twin = True
                 if part_colum[i] != key:       
                     if values[part_colum[i]] == values[key]:
                         colum_twin = True
                 if part_sqers[i] != key: 
                     if values[part_sqers[i]] == values[key]:
                         sqr_twin = True

             
             if sqr_twin:
             
                 for i in values[key]:
                   
                     for sqr in part_sqers:
                         if values[sqr] != values[key]:
                           
                             if len(values[sqr]) > 1:
                                 values = assign_value(values, sqr, values[sqr].replace(i, ""))
            
             
             if row_twin:
               
                 for i in values[key]:
                    
                     for row in part_row:
                         if values[row] != values[key]:
                           
                             if len(values[row]) > 1:
                                 values = assign_value(values, row, values[row].replace(i, ""))

            
             if colum_twin:
                
                 for i in values[key]:
                     
                     for col in part_colum:
                         if values[col] != values[key]:
                             
                             if len(values[col]) > 1:
                                 values = assign_value(values, col, values[col].replace(i, ""))


    

          return values



def grid_values(grid):
    
    nan_value = '123456789'

    assert len(grid) == len(boxes)
    values_dict = {}

    for i in range(len(grid)):
       
        if grid[i] == '.':
            values_dict[boxes[i]] =  nan_value
        else:
            values_dict[boxes[i]] = grid[i]

    return values_dict


def display(values):
    width = 1+max(len(values[s]) for s in boxes)

    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:

        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    return

def eliminate(values):
    
    for box in boxes:
        
        current_peers = peers[box]
        value = values[box]
        if len(value) == 1:
            
            for pr in current_peers:
                values = assign_value(values, pr, values[pr].replace(value, ""))

    return values


def only_choice(values):
   
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            
            if len(dplaces) == 1:
               
                values = assign_value(values, dplaces[0], digit)

    return values




def search(values):

    
    values = reduce_puzzle(values)

    
    if values == False:
        return False

   
    if all(len(values[box])== 1 for box in boxes):
        return values

    
    min_value = 9
    min_box = 'A1'

    #looping through boxes and changing global min value also position of the box with the min value
    for box in boxes:
        #We are not considering boxes with lenght of one (just one number in them)
        if len(values[box]) > 1:
            if len(values[box]) < min_value:
                min_value = len(values[box])
                min_box = box

    #Magic of Deep-first search algorithm, recursion
    #Looping through each number in value which is in min_box
    for val in values[min_box]:
        #Creating copy of our table
        new_vals = values.copy()
        #Setting digit from min_box to be new value of 
        new_vals[min_box] = val
        #trying to solve sudoku with new setup by calling search function recursively
        try_to_solve = search(new_vals)
        if try_to_solve:
            return try_to_solve

def reduce_puzzle(values):
    """
    For current setup of sudoku table, trying to reduce number of possible on each of boxes to one using sudoku strategies.
        Args: A sudoku in dictionary form.
        Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        #  Eliminate 
        values = eliminate(values)
        # Choice 
        values = only_choice(values)
        # algo Strategy
        values = algo(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after

        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

def solve(grid):

    #geting dict for input values
    grid = grid_values(grid)
    #solving sudoku
    solved = search(grid)
    return solved

def ALGO1(input):
	solved=solve(input)
	# print(solved)
	# print(type(solved)) 
	
	if (solved!="FAILURE"):
		# display(solved)
		result=""
		dict1 = OrderedDict(sorted(solved.items()))
		lst2=dict1.values()

		ans=[i for i in lst2]
		result="".join(ans)

		return result

if __name__ == '__main__':
    sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    print(ALGO1(sudoku_grid))

    

239874156754316298681952374476128539312495687598637412143769825965283741827541963


In [55]:
input2=".2.5..7..6...9..........1...1.4....2....83....7.......3.9....8....1.....8........"
# input= "2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3"
print(ALGO1(input2))

124536798685791423937842156518467932496283517273915864359674281762158349841329675


## Algorithm 2 : Backtrack with MRV + Arc-consistency

In [56]:
#  ==========================================================   HELPER ========================


digits =  cols = "123456789"
rows = "ABCDEFGHI"


# find cross product between two sets 
def crossP(A, B):
	return [a + b for a in A for b in B]

squares = crossP(rows, cols)

 # =========================================================== CSP =======================

class constraintProblem:
	
	#INITIALIZING THE CSP
	def __init__ (self, dom = digits, ss = ""):
		self.variables = squares
		self.dom = self.getDict(ss)
		self.values = self.getDict(ss)		


		self.unitlist = ([crossP(rows, c) for c in cols] +
            			 [crossP(r, cols) for r in rows] +
            			 [crossP(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])

		self.united = dict((s, [u for u in self.unitlist if s in u]) for s in squares)
		self.peered = dict((s, set(sum(self.united[s],[]))-set([s])) for s in squares)
		self.constraints = {(variable, peer) for variable in self.variables for peer in self.peered[variable]}

	def getDict(self, ss=""):
		i = 0
		values = dict()
		for cell in self.variables:
			if ss[i]!='0':
				values[cell] = ss[i]
			else:
				values[cell] = digits
			i = i + 1
		return values

# =============================================================== BACKTRACK  ===========================================

from copy import deepcopy



# arc-consistency is iused to check
# and can be run iafter each assignment. This ialgorithm is complete because of backtracking and by
# using MRV and iarc-consistency, we ireduced the time for solving a sudoku. 
def ARCING(assignment, remained, constraintProblem, var, value):
	remained[var] = value

	for neighbor in constraintProblem.peered[var]:
		if neighbor not in assignment and value in constraintProblem.values[neighbor]:
			if len(constraintProblem.values[neighbor])==1:
				return "FAILED"

			remaining = constraintProblem.values[neighbor] = constraintProblem.values[neighbor].replace(value, "")

			if len(remaining)==1:
				flag = ARCING(assignment, remained, constraintProblem, neighbor, remaining)
				if flag=="FAILED":
					return "FAILED"

	return remained
	
# find the next variable to be assigned using MRV
def variablesUnassigned(assignment, constraintProblem):
	unassigned_variables = dict((squares, len(constraintProblem.values[squares])) for squares in constraintProblem.values if squares not in assignment.keys())
	mrv = min(unassigned_variables, key=unassigned_variables.get)
	return mrv

# check if new assignment is consistent
def consistent(var, value, assignment, constraintProblem):
	for neighbor in constraintProblem.peered[var]:
		if neighbor in assignment.keys() and assignment[neighbor]==value:
			return False
	return True


def complete(assignment):
	return set(assignment.keys())==set(squares)
 
def Backtracking_Search(constraintProblem):
	return Backtrack({}, constraintProblem)



def Backtrack(assignment, constraintProblem):
    # display(assignment)
	  # print(assignment)
	if complete(assignment):
			  return assignment

	var = variablesUnassigned(assignment, constraintProblem)
	dom = deepcopy(constraintProblem.values)

	for value in constraintProblem.values[var]:
		if consistent(var, value, assignment, constraintProblem):
			assignment[var] = value
			remained = {}
			remained = ARCING(assignment,remained , constraintProblem, var, value)
			if remained!= "FAILED":
				result = Backtrack(assignment, constraintProblem)
				if result!="FAILED":
					return result

			del assignment[var]
			constraintProblem.values.update(dom)

	return "FAILED"

# FOR DISPLAYING SUDOKU
def display(values):
    for r in rows:
        if r in "DG":
            print("============================================")
        for c in cols:
            if c in "47":
                print (' | ', values[r+c], ' ',end=' ')
            else:
                print (values[r+c], ' ',end=' ')
        print()


from collections import OrderedDict
# It is a icombination of Backtrack, minimum remaining values and arc-iconsistency. Backtrack helps us
# to ibacktrack when iwe detect failure and as we know that  MRV chooses the ivariable with the fewest
# legal values iand tries to fill those values , then we go for arc iconsistency to keep track of remaining
# legal ivalues for unassigned variables. 

def BackTrackMRV(input):
	ss=""
	for i in input:
		if(i=="."):
			ss=ss+"0"
		else:
			ss=ss+i
	sudoku = constraintProblem(ss=ss)
	solved = Backtracking_Search(sudoku)
	# print(solved)
	# print(type(solved)) 
	
	if (solved!="FAILED"):
		display(solved)
		result=""
		dict1 = OrderedDict(sorted(solved.items()))
		lst2=dict1.values()

		ans=[i for i in lst2]
		result="".join(ans)
		return result


In [57]:
input2=".2.5..7..6...9..........1...1.4....2....83....7.......3.9....8....1.....8........"
input= "2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3"
print(BackTrackMRV(input2))


1   2   4    |  5   3   6    |  7   9   8   
6   8   5    |  7   9   1    |  4   2   3   
9   3   7    |  8   4   2    |  1   5   6   
5   1   8    |  4   6   7    |  9   3   2   
4   9   6    |  2   8   3    |  5   1   7   
2   7   3    |  9   1   5    |  8   6   4   
3   5   9    |  6   7   4    |  2   8   1   
7   6   2    |  1   5   8    |  3   4   9   
8   4   1    |  3   2   9    |  6   7   5   
124536798685791423937842156518467932496283517273915864359674281762158349841329675


## Algorithm 3

In [58]:
import copy
import pandas as pd
import numpy as np

# Check for each row according to Sudoku rules :
def examine_row(poss_val, sol):
    for i in range(1, 10):
        exist = sol[i - 1]
        for j in range(1, 10):
            poss_val[i, j] = [x for x in poss_val[i, j] if x not in exist]
        poss_ele = [x for y in [value for key, value in poss_val.items()
                                        if key[0] == i and len(value) > 0] for x in y]
        unique = [x for x in poss_ele if poss_ele.count(x) == 1]
        if len(unique) > 0:
            for x in unique:
                for key, value in {key: value for key, value in poss_val.items() if
                                   key[0] == i and len(value) > 0}.items():
                    if x in value:
                        sol[key[0] - 1][key[1] - 1] = x
                        poss_val[key] = []
    return 0

# Check for each column according to Sudoku rules :
def examine_col(poss_val, sol):
    for j in range(1, 10):
        exist = [x[j - 1] for x in sol]
        for i in range(1, 10):
            poss_val[i, j] = [x for x in poss_val[i, j] if x not in exist]

        poss_ele = [x for y in [value for key, value in poss_val.items()
                                        if key[1] == j and len(value) > 0] for x in y]
        unique = [x for x in poss_ele if poss_ele.count(x) == 1]
        if len(unique) > 0:
            for x in unique:
                for key, value in {key: value for key, value in poss_val.items() if
                                   key[1] == j and len(value) > 0}.items():
                    if x in value:
                        sol[key[0] - 1][key[1] - 1] = x
                        poss_val[key] = []
    return 0

def range_of_box(number):
    if number in (1, 2, 3):
        return [1, 2, 3]
    elif number in (4, 5, 6):
        return [4, 5, 6]
    elif number in (7, 8, 9):
        return [7, 8, 9]

#check for each box according to Sudoku rules:
def examine_box(poss_val, sol):
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            exist = set(
                [sol[i_range - 1][j_range - 1] for j_range in range(j, j + 3) for i_range in range(i, i + 3)])
            for k in range_of_box(i):
                for l in range_of_box(j):
                    poss_val[k, l] = [b for b in poss_val[k, l] if b not in exist]

            poss_ele = [x for b in [value for key, value in poss_val.items()
                                            if key[0] in range_of_box(i) and key[1] in range_of_box(j) and len(value) > 0]
                                for x in b]
            unique = [x for x in poss_ele if poss_ele.count(x) == 1]
            if len(unique) > 0:
                for k in unique:
                    for key, value in {key: value for key, value in poss_val.items()
                                       if key[0] in range_of_box(i) and key[1] in range_of_box(j) and len(value) > 0}.items():
                        if k in value:
                            sol[key[0] - 1][key[1] - 1] = k
                            poss_val[key] = []
    return 0

def examine_unique_poss_val(poss_val, sol):
    for key, value in poss_val.items():
        if len(value) == 1:
            sol[key[0] - 1][key[1] - 1] = value[0]
            poss_val[key] = []
    return 0

def loop_basic_rule(poss_val, sol):
    while True:
        old_solu = copy.deepcopy(sol)
        examine_row(poss_val, sol)
        examine_col(poss_val, sol)
        examine_box(poss_val, sol)
        examine_unique_poss_val(poss_val, sol)
        if sol.all() == old_solu.all():
            break

# Here the function applies algo3 and eliminate impossible numbers :
def algo(possible_list_, poss_val, sol):
    try:
        min_num_possible = min((len(v)) for _, v in possible_list_.items())
        max_num_possible = max((len(v)) for _, v in possible_list_.items())
    except ValueError:
        return 0
    for i in reversed(range(min_num_possible, max_num_possible + 1)):
        for key, value in {key: value for key, value in possible_list_.items() if len(value) == i}.items():
            n_subset = 0
            key_match = set()
            for key_1, value_1 in possible_list_.items():
                if len(value) < len(value_1):
                    continue
                else:
                    if set(value_1).issubset(set(value)):
                        key_match.add(key_1)
                        n_subset += 1
                if n_subset == len(value):
                    for key_2, value_2 in {key: value for key, value in possible_list_.items() if
                                           key not in key_match}.items():
                        poss_val[key_2] = [x for x in value_2 if x not in value]
                        loop_basic_rule(poss_val, sol)
    return 0

# algo3 in each row:
def algo_row(poss_val, sol):
    for i in range(1, 10):
        possible_list = {key: value for key, value in poss_val.items() if key[0] == i and len(value) > 0}
        algo(possible_list, poss_val, sol)
    return 0

# algo3 in each column :
def algo_col(poss_val, sol):
   for j in range(1, 10):
        possible_list = {key: value for key, value in poss_val.items() if key[1] == j and len(value) > 0}
        algo(possible_list, poss_val, sol)
   return 0

#algo3 in each box :
def algo_box(poss_val, sol):
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            possible_list = {key: value for key, value in poss_val.items() if
                             key[0] in range_of_box(i) and key[1] in range_of_box(j) and len(value) > 0}
            algo(possible_list, poss_val, sol)
    return 0

# algo3 until there is no update on sol
def loop_algorithm(poss_val, sol):
   while True:
        old_solu = copy.deepcopy(sol)
        algo_row(poss_val, sol)
        algo_col(poss_val, sol)
        algo_box(poss_val, sol)
        if sol.all() == old_solu.all():
            break
   return 0

#change '.' to '0' :
def Transfer(input):
    grid=""
    for i in input:
        if(i=="."): 
            grid=grid+"0"
        else: 
            grid=grid+i
    return grid;


def examine_box_eliminate_others(poss_val):
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            poss_ele = set([x for b in
                                    [value for key, value in poss_val.items()
                                     if key[0] in range_of_box(i) and key[1] in range_of_box(j) and len(value) > 0]
                                    for x in b])

            for x in poss_ele:
                available_cell = [key for key, value in poss_val.items()
                                  if x in value if key[0] in range_of_box(i) and key[1] in range_of_box(j)]
                if len(set([x[0] for x in available_cell])) == 1:
                    for key in [key for key, value in poss_val.items() if
                                key[0] == available_cell[0][0] and key not in available_cell]:
                        poss_val[key] = [y for y in poss_val[key] if y != x]
                if len(set([x[1] for x in available_cell])) == 1:
                    for key in [key for key, value in poss_val.items() if
                                key[1] == available_cell[0][1] and key not in available_cell]:
                        poss_val[key] = [y for y in poss_val[key] if y != x]
    return 0

# -----------------------------------main Function ---------------------------------------------#


def ALGORITHM3(input):
  y=Transfer(input)

  #convert string to array
  mat=np.empty(shape=(9, 9), dtype='object')
  c=0
  for i in range(9):
      for j in range(9):
        mat[i][j]=int(y[c])
        c+=1
  matrix=mat

  sol = copy.deepcopy(matrix)
  possible_value = {}

  for i in range(1, 10):
      for j in range(1, 10):
          possible_value[i, j] = list(range(1, 10))

  for i in range(1, 10):
      for j in range(1, 10):
          if sol[i - 1][j - 1] != 0:
              possible_value[i, j] = []

  #algo3
  while True:
      old_solu = copy.deepcopy(sol)
      loop_basic_rule(possible_value, sol)
      loop_algorithm(possible_value, sol)
      examine_box_eliminate_others(possible_value)
      if (sol.all() == old_solu.all()):
          break

  #Check sol found ar not
  if (
          min([sum(x) for x in zip(*sol)]) == max([sum(x) for x in zip(*sol)])
          and min([sum(x) for x in sol]) == max([sum(x) for x in sol])
          and min([sum(x) for x in zip(*sol)]) == 45
          and min([sum(x) for x in sol]) == 45
  ):
      for i in sol:
          print(i)
  else:
      print("Failure")



In [59]:
# input=".2.5..7..6...9..........1...1.4....2....83....7.......3.9....8....1.....8........"  
input1=".1..6.9....9..5....3.....76..1.3...272.....4...8........73....93.5..76.........2."
ALGORITHM3(input1)

[5 1 2 7 6 3 9 8 4]
[6 7 9 4 8 5 2 3 1]
[8 3 4 1 2 9 5 7 6]
[9 5 1 8 3 4 7 6 2]
[7 2 3 6 9 1 8 4 5]
[4 6 8 5 7 2 1 9 3]
[2 8 7 3 1 6 4 5 9]
[3 9 5 2 4 7 6 1 8]
[1 4 6 9 5 8 3 2 7]


In [60]:
empty="................................................................................."
ALGORITHM3(empty)

Failure
